<a href="https://colab.research.google.com/github/ArpitaDeshmukh2024/SPPU_DL_practicals/blob/main/DL4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import kagglehub
import matplotlib.pyplot as plt

In [21]:
path = kagglehub.dataset_download("akram24/google-stock-price-test")
dataset_path = f"{path}/Google_Stock_Price_Test.csv"

In [22]:
data = pd.read_csv(dataset_path, usecols = [1]).dropna().values.astype(float)

In [23]:
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [24]:
def create_dataset(dataset, time_step = 10):
  x, y = [], []
  for i in range(len(dataset) - time_step):
    x.append(dataset[i:i + time_step, 0])
    y.append(dataset[i + time_step, 0])
  return np.array(x), np.array(y)

In [26]:
time_step = min(10, len(data) - 1)
if len(data) > time_step:
  x , y = create_dataset(data, time_step)
  x = x.reshape(x.shape[0], x.shape[1], 1)

  xtrain, xtest, ytrain, ytest = train_test_split(x , y, test_size = 0.2, random_state = 42)

  model = Sequential([
      SimpleRNN(50, activation = "relu", return_sequences = True, input_shape = (time_step, 1)),
      SimpleRNN(50, activation = "relu"),
      Dense(1)
  ])

  model.compile(optimizer = "adam", loss = "mse")
  model.fit(xtrain, ytrain, epochs = 20, batch_size = 32, validation_data = (xtest, ytest))

  print("Test Loss : ", model.evaluate(xtest, ytest))

else:
  print("Error : Not enough data points to create sequences. Consider using a smaller time step")

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.5503 - val_loss: 0.2298
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step - loss: 0.4632 - val_loss: 0.1780
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.3969 - val_loss: 0.1413
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.3472 - val_loss: 0.1088
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.3030 - val_loss: 0.0814
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.2623 - val_loss: 0.0626
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.2277 - val_loss: 0.0431
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.1953 - val_loss: 0.0252
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.1645 - val_loss: 0.0123
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.1360 - val_loss: 0.0050
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.1100 - val_loss: 0.0033
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.0886 - val_loss: 0.0086
Epoch 13